# ML Pipeline for Accelerometer Data

First just dabble with accelerometer data.

We will use it for seizure detection, and then seizure prediction. 

Using autoML approach plus feature set (either TSFresh or pycatch22)

In [1]:
# Import Packages
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import io 
import re
from scipy.fft import fft, fftfreq
from pathlib import Path

import pycatch22

In [2]:
# First load one of the Pickle Files of the Patient

file_path = Path('/Users/pokhims/Documents/Coding/PK_EForecasting/Ignore/Output/1110_model_data_0.pkl')

data = pd.read_pickle(file_path)

In [3]:
file_path

PosixPath('/Users/pokhims/Documents/Coding/PK_EForecasting/Ignore/Output/1110_model_data_0.pkl')

In [4]:
data

,time_of_day,acc_mag_fft,bvp_fft,eda_fft,hr_fft,temp_fft,acc_quality,bvp_quality,eda_quality,seizure_ind,label
0,23,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0.0,0.0
1,23,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0.0,0.0
2,23,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0.0,0.0
3,23,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0.0,0.0
4,23,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
76799995,4,0.037673,0.004349,0.111067,0.025424,0.110821,2.232677,1,1,0.0,0.0
76799996,4,0.050315,0.005017,0.171193,0.030320,0.213742,2.232677,1,1,0.0,0.0
76799997,4,0.040327,0.001360,0.177507,0.053572,0.305417,2.232677,1,1,0.0,0.0
76799998,4,0.056146,0.000919,0.178095,0.063388,0.393353,2.232677,1,1,0.0,0.0


In [5]:
#data["acc_mag_fft"].plot()

In [6]:
data["acc_mag_fft"].max()

4.680402350167861

In [7]:
data.shape[0]

76800000

In [10]:
def window_catch22(data,window_size=10,f_s=128):
    """
    Runs catch-22 on a given input.  The input is presumably an entire 10min file, but is certainly a pandas data frame.

    f_s = 128Hz by default (as per https://eval.ai/web/challenges/challenge-page/1693/overview)
    window_size = 10s by default.  
    """

    # Figure out how many elements goes into each window.
    number = int(window_size*f_s)

    # Length of the data_file
    file_len = data.shape[0]

    # Initialize list to save data per window
    li = []

    # Loop through the data file, creating windows of data and running Catch22.
    for k in np.arange(np.floor(file_len/number)):

        # Create or specify the window of data.
        # Just in case, maybe the 10min data file is completely empty or something, we need to do a try-except to move on.
        try:
            # Specify window
            window = data.iloc[int(k*number):int((k+1)*number),:]
        except:
            continue

        # Basic checks which might throw errors.  If they do, we just skip that window.
        # If array is empty skip it.
        if window.size==0:
            continue
        # If array has Nans skip it.  - This is not ideal, but just want to get something going quickly. 
        elif np.isnan(window).any().any():
            continue

        # For each column of dataframe apply catch22
        li1 = []
        for i in np.arange(data.shape[1]):
            features = pycatch22.catch22_all(window[i], catch24=True, )
            li1.append(features["values"])  

        # Turn this list into an array 
        li1 = np.array(li1)
        # Add to list saving data per window
        li.append(li1)

    # Turn the list into a 3D array
    frame = np.array(li) 
    
    return frame   

In [11]:
window_catch22(data)

KeyError: 0

In [13]:
window_size = 10
f_s = 128

# Specify columns to use
cols = ["acc_mag_fft","bvp_fft","eda_fft","temp_fft","hr_fft"]

# Figure out how many elements goes into each window.
number = int(window_size*f_s)

# Length of the data_file
file_len = data.shape[0]

# Initialize list to save data per window
li = []

# Loop through the data file, creating windows of data and running Catch22.
for k in np.arange(np.floor(file_len/number)):

    # Create or specify the window of data.
    # Just in case, maybe the 10min data file is completely empty or something, we need to do a try-except to move on.
    try:
        # Specify window
        window = data.iloc[int(k*number):int((k+1)*number),:]
    except:
        continue

    # Basic checks which might throw errors.  If they do, we just skip that window.
    # If array is empty skip it.
    if window.size==0:
        continue
    # If array has Nans skip it.  - This is not ideal, but just want to get something going quickly. 
    elif np.isnan(window).any().any():
        continue

    # For each column of dataframe apply catch22
    li1 = []
    for i in np.arange(data.shape[1]):
        features = pycatch22.catch22_all(window[i], catch24=True, )
        li1.append(features["values"])  

    # Turn this list into an array 
    li1 = np.array(li1)
    # Add to list saving data per window
    li.append(li1)

# Turn the list into a 3D array
frame = np.array(li) 

KeyError: 0

In [17]:
window.iloc[:,0]

38400    23
38401    23
38402    23
38403    23
38404    23
         ..
39675    23
39676    23
39677    23
39678    23
39679    23
Name: time_of_day, Length: 1280, dtype: int64